This demonstration is based off the companion notebook for the book [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff). This notebook was generated for TensorFlow 2.6, but is current up to 2.12.0.

In [1]:
import tensorflow as tf
import numpy as np
import time
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.metrics import AUC

print(tf.__version__)

2023-07-27 22:18:53.781543: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.12.0


## Demo Part I - Add TensorBoard to a model 
### The IMDB dataset
### ML problem: Binary classification, positive or negative reviews?

**Loading the IMDB dataset**

In [2]:
from tensorflow.keras.datasets import imdb

# Load training and test data from imdb dataset
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(
    num_words=10000)

# Values of text from this dataset integer encoded; this first sample
# train_data[0]

# The label for the first sample (1 or 0)
# train_labels[0]

# The number of trainign samples 1000 total
# max([max(sequence) for sequence in train_data])

17464789/17464789 [==============================] - 0s 0us/step


**Decoding reviews back to text**

In [3]:
# If curious about the decoded text, set DECODE to True and run.
# Only the first sample is decoded.
DECODE = True
if DECODE:
        
    word_index = imdb.get_word_index()
    reverse_word_index = dict(
        [(value, key) for (key, value) in word_index.items()])
    decoded_review = " ".join(
        [reverse_word_index.get(i - 3, "?") for i in train_data[0]])

    # See the decoded version of the encoded words from the imdb dataset
    print(decoded_review)

1641221/1641221 [==============================] - 0s 0us/step
? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amaz

### Preparing the data

**Encoding the integer sequences via multi-hot encoding**

In [4]:

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        for j in sequence:
            results[i, j] = 1.
    return results
# create train and test datasets    
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

# See multi-hot encoded vectorrized sequence
# x_train[0]


In [5]:
# Cast as floats for training
y_train = np.asarray(train_labels).astype("float32")
y_test = np.asarray(test_labels).astype("float32")

# Create train/validation split
x_val = x_train[:10000]
y_val = y_train[:10000]
partial_x_train = x_train[10000:]
partial_y_train = y_train[10000:]

### Building your model using TensorBoard

In [6]:
# setup logging scheme
MODEL = "imdb-model-{}".format(int(time.time()))

# initialize callback (create logs directory first or model.fit will fail)
tbcallback = TensorBoard(log_dir = "logs/{}".format(MODEL), histogram_freq=1)
# windows?
# tbcallback = TensorBoard(log_dir = "logs\{}".format(MODEL), histogram_freq=1)

print(MODEL + " <- this will be your TensorBoard log file name.")

imdb-model-1690496345 <- this will be your TensorBoard log file name.


In [7]:
# initialize a sequential fully connected network to address the ML problem
model = keras.Sequential([
    layers.Dense(16, activation="relu", name="L1_16_Dense_RELU"),
    layers.Dense(16, activation="relu", name="L2_16_Dense_RELU"),
    layers.Dense(1, activation="sigmoid", name="L3_1_Dense_SIG")
])

# Compile the network using your optmizier, loss, and metrics
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

# Fit your model, including TensorBoard in the callback list
model.fit(partial_x_train,
          partial_y_train, 
          validation_data=(x_val, y_val),
          epochs=20, 
          batch_size=512, 
          callbacks=[tbcallback])
            

2023-07-27 22:19:06.821024: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10481 MB memory:  -> device: 0, name: NVIDIA TITAN V, pci bus id: 0000:88:00.0, compute capability: 7.0


Epoch 1/20


2023-07-27 22:19:10.772942: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f2738cedc20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-07-27 22:19:10.773000: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA TITAN V, Compute Capability 7.0
2023-07-27 22:19:10.778491: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-07-27 22:19:11.023264: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-07-27 22:19:11.170305: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


30/30 [==============================] - 5s 79ms/step - loss: 0.5123 - accuracy: 0.7807 - val_loss: 0.3920 - val_accuracy: 0.8559
Epoch 2/20
30/30 [==============================] - 1s 32ms/step - loss: 0.3146 - accuracy: 0.8928 - val_loss: 0.3071 - val_accuracy: 0.8824
Epoch 3/20
30/30 [==============================] - 1s 28ms/step - loss: 0.2373 - accuracy: 0.9186 - val_loss: 0.2829 - val_accuracy: 0.8888
Epoch 4/20
30/30 [==============================] - 1s 27ms/step - loss: 0.1916 - accuracy: 0.9361 - val_loss: 0.2759 - val_accuracy: 0.8878
Epoch 5/20
30/30 [==============================] - 1s 27ms/step - loss: 0.1590 - accuracy: 0.9478 - val_loss: 0.2785 - val_accuracy: 0.8880
Epoch 6/20
30/30 [==============================] - 1s 27ms/step - loss: 0.1374 - accuracy: 0.9529 - val_loss: 0.2939 - val_accuracy: 0.8847
Epoch 7/20
30/30 [==============================] - 1s 27ms/step - loss: 0.1177 - accuracy: 0.9633 - val_loss: 0.3035 - val_accuracy: 0.8849
Epoch 8/20
30/30 [======

### Review TensorBoard output to evaluate how model performance.

In [8]:
# In console, from directory containing the "logs" directory.
# tensorboard --logdir logs

### Refit model with adjusted parameters and all training data.

In [9]:
# Make any adjustments for your final model
model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

# added in a Precision Recall curve metric
metrics=[tf.keras.metrics.Accuracy(name='accuracy'),
        tf.keras.metrics.AUC(name='prc', curve='PR')]


# Compile the network using your optmizier, loss, and metrics
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=metrics)

# Update logging scheme
MODEL = "imdb-final-model-{}".format(int(time.time()))
tbcallback = TensorBoard(log_dir = "logs/{}".format(MODEL), histogram_freq=1)
# tbcallback = TensorBoard(log_dir = "logs\{}".format(MODEL), histogram_freq=1)

# Refit your final model
model.fit(x_train,                              # No longer partial_x_train
          y_train,                              # No longer partial_y_train
          epochs=5,                             # Changed to 5 epochs 
          batch_size=512, 
          callbacks=[tbcallback])
            

Epoch 1/5
49/49 [==============================] - 3s 18ms/step - loss: 0.4601 - accuracy: 0.0000e+00 - prc: 0.8953
Epoch 2/5
49/49 [==============================] - 1s 16ms/step - loss: 0.2694 - accuracy: 4.0000e-05 - prc: 0.9637
Epoch 3/5
49/49 [==============================] - 1s 16ms/step - loss: 0.2112 - accuracy: 4.0000e-05 - prc: 0.9759
Epoch 4/5
49/49 [==============================] - 1s 14ms/step - loss: 0.1796 - accuracy: 4.4000e-04 - prc: 0.9818
Epoch 5/5
49/49 [==============================] - 1s 12ms/step - loss: 0.1581 - accuracy: 0.0012 - prc: 0.9855


In [10]:

# Check how accurate the model is at predicting unseen observations
results = model.evaluate(x_test, y_test)

# A tuple containing loss, accuracy
results

782/782 [==============================] - 2s 3ms/step - loss: 0.2975 - accuracy: 4.0000e-04 - prc: 0.9459


[0.29754045605659485, 0.00039999998989515007, 0.9458764791488647]

### Using a trained model to generate predictions on new data

In [11]:
model.predict(x_test)

782/782 [==============================] - 2s 2ms/step


array([[0.18177032],
       [0.9999422 ],
       [0.8168049 ],
       ...,
       [0.0925273 ],
       [0.04993724],
       [0.5770854 ]], dtype=float32)

# Part 2: Comparing multiple models

## Classifying newswires: A multiclass classification example

### The Reuters dataset

In [12]:
import tensorflow as tf
import numpy as np
import time
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.datasets import reuters

**Loading the Reuters dataset**

In [13]:
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(
    num_words=10000)

#len(train_data)
#len(test_data)
#train_data[10]

2110848/2110848 [==============================] - 0s 0us/step


**Decoding newswires back to text**

In [14]:
word_index = reuters.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_newswire = " ".join([reverse_word_index.get(i - 3, "?") for i in
    train_data[0]])

#train_labels[10]

550378/550378 [==============================] - 0s 0us/step


### Preparing the data

**Encoding the input and labels & Splitting data**

In [15]:
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        for j in sequence:
            results[i, j] = 1.
    return results

#Encode input
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

#Encode labels
def to_one_hot(labels, dimension=46):
    results = np.zeros((len(labels), dimension))
    for i, label in enumerate(labels):
        results[i, label] = 1.
    return results
y_train = to_one_hot(train_labels)
y_test = to_one_hot(test_labels)

# import 1-hot encoding library
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(train_labels)
y_test = to_categorical(test_labels)

#Set aside validation data
x_val = x_train[:1000]
partial_x_train = x_train[1000:]
y_val = y_train[:1000]
partial_y_train = y_train[1000:]

### Building your model

**Model definition**

In [16]:
# Create two models, the only difference is hidden layer activations

model1 = keras.Sequential([
    layers.Dense(64, activation="tanh"),
    layers.Dense(64, activation="tanh"),
    layers.Dense(46, activation="softmax")
])

model2 = keras.Sequential([
    layers.Dense(64, activation="relu"),
    layers.Dense(64, activation="relu"),
    layers.Dense(46, activation="softmax")
])

**Compiling the model**

In [17]:
model1.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

model2.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

**Training the model**

In [18]:
# setup logging scheme for model1
MODEL = "mod1-reuters-{}".format(int(time.time()))
tbcallback = TensorBoard(log_dir = "logs/{}".format(MODEL), histogram_freq=1)
#tbcallback = TensorBoard(log_dir = "logs\{}".format(MODEL), histogram_freq=1)

history1 = model1.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    callbacks=[tbcallback])

MODEL = "mod2-reuters-{}".format(int(time.time()))
tbcallback = TensorBoard(log_dir = "logs/{}".format(MODEL), histogram_freq=1)
#tbcallback = TensorBoard(log_dir = "logs\{}".format(MODEL), histogram_freq=1)

history2 = model2.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    callbacks=[tbcallback])

Epoch 1/20
16/16 [==============================] - 3s 62ms/step - loss: 2.4063 - accuracy: 0.5263 - val_loss: 1.6900 - val_accuracy: 0.6220
Epoch 2/20
16/16 [==============================] - 1s 45ms/step - loss: 1.4525 - accuracy: 0.6840 - val_loss: 1.3365 - val_accuracy: 0.6970
Epoch 3/20
16/16 [==============================] - 1s 44ms/step - loss: 1.1204 - accuracy: 0.7635 - val_loss: 1.1352 - val_accuracy: 0.7650
Epoch 4/20
16/16 [==============================] - 1s 41ms/step - loss: 0.8966 - accuracy: 0.8260 - val_loss: 1.0182 - val_accuracy: 0.7860
Epoch 5/20
16/16 [==============================] - 1s 39ms/step - loss: 0.7295 - accuracy: 0.8623 - val_loss: 0.9301 - val_accuracy: 0.8070
Epoch 6/20
16/16 [==============================] - 1s 41ms/step - loss: 0.5992 - accuracy: 0.8860 - val_loss: 0.8783 - val_accuracy: 0.8190
Epoch 7/20
16/16 [==============================] - 1s 42ms/step - loss: 0.4968 - accuracy: 0.9064 - val_loss: 0.8469 - val_accuracy: 0.8220
Epoch 8/20
16

**Retraining a model from scratch**

In [19]:
MODEL = "final-reuters-{}".format(int(time.time()))
tbcallback = TensorBoard(log_dir = "logs/{}".format(MODEL), histogram_freq=1)
#tbcallback = TensorBoard(log_dir = "logs\{}".format(MODEL), histogram_freq=1)

model = keras.Sequential([
  layers.Dense(64, activation="relu"),
  layers.Dense(64, activation="relu"),
  layers.Dense(46, activation="softmax")
])
model.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

model.fit(x_train,
          y_train,
          epochs=20,
          batch_size=512,
          callbacks=[tbcallback])
          
results = model.evaluate(x_test, y_test)


Epoch 1/20
18/18 [==============================] - 2s 37ms/step - loss: 2.6620 - accuracy: 0.5163
Epoch 2/20
18/18 [==============================] - 1s 40ms/step - loss: 1.4816 - accuracy: 0.6820
Epoch 3/20
18/18 [==============================] - 1s 33ms/step - loss: 1.1367 - accuracy: 0.7509
Epoch 4/20
18/18 [==============================] - 1s 35ms/step - loss: 0.9262 - accuracy: 0.7976
Epoch 5/20
18/18 [==============================] - 1s 39ms/step - loss: 0.7662 - accuracy: 0.8340
Epoch 6/20
18/18 [==============================] - 1s 36ms/step - loss: 0.6389 - accuracy: 0.8588
Epoch 7/20
18/18 [==============================] - 1s 34ms/step - loss: 0.5301 - accuracy: 0.8828
Epoch 8/20
18/18 [==============================] - 1s 34ms/step - loss: 0.4493 - accuracy: 0.9019
Epoch 9/20
18/18 [==============================] - 1s 35ms/step - loss: 0.3755 - accuracy: 0.9176
Epoch 10/20
18/18 [==============================] - 1s 34ms/step - loss: 0.3189 - accuracy: 0.9282
Epoch 11/

In [20]:
results

[1.0026326179504395, 0.8014247417449951]

In [21]:
import copy
test_labels_copy = copy.copy(test_labels)
np.random.shuffle(test_labels_copy)
hits_array = np.array(test_labels) == np.array(test_labels_copy)
hits_array.mean()

0.17764915405164738

### Generating predictions on new data

In [22]:
predictions = model.predict(x_test)

71/71 [==============================] - 0s 2ms/step


In [23]:
predictions[0].shape

(46,)

In [24]:
np.sum(predictions[0])

0.99999994

In [25]:
np.argmax(predictions[0])

3